In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/weatherpy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,hithadhoo,MV,84.42,broken clouds,-0.6000,73.0833,Scoop Guest House
1,2,punta arenas,CL,39.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
2,4,rikitea,PF,76.77,scattered clouds,-23.1203,-134.9692,People ThankYou
3,5,petropavlovsk-kamchatskiy,RU,39.90,overcast clouds,53.0452,158.6483,The Hotel Petropavlovsky
4,6,tasiilaq,GL,35.76,overcast clouds,65.6145,-37.6368,The Red House


In [4]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'oskaloosa']
vacation_end = vacation_df.loc[vacation_df['City'] == 'bethel']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'decatur']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'hamilton'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'jamestown'] 
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,24,jamestown,US,50.95,clear sky,42.097,-79.2353,Hampton Inn & Suites Jamestown


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.iloc[0,5:7].to_numpy()
end = vacation_end.iloc[0,5:7].to_numpy()
stop1 = vacation_stop1.iloc[0,5:7].to_numpy()
stop2 = vacation_stop2.iloc[0,5:7].to_numpy()
stop3 = vacation_stop3.iloc[0,5:7].to_numpy()

start

array([41.2964, -92.6444], dtype=object)

In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure(center=(38, -122), zoom_level = 7)
oskaloosa = start
decatur = stop1
hamilton = stop2
jamestown = stop3 
roadtrip = gmaps.directions_layer(oskaloosa, oskaloosa, 
                                  waypoints=[decatur, hamilton, jamestown],
                                  travel_mode="DRIVING")

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3]
                         ,ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,449,oskaloosa,US,59.41,clear sky,41.2964,-92.6444,EverSpring Inn & Suites
1,362,decatur,US,56.98,clear sky,39.8403,-88.9548,Haworth Cottage
2,300,hamilton,US,55.26,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast
3,24,jamestown,US,50.95,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(marker_layer)
fig.add_layer(roadtrip)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))